In [ ]:
from kafka import KafkaConsumer
import pandas as pd
from tqdm import tqdm
import uuid
import time
from collections import defaultdict, deque
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from apscheduler.schedulers.background import BackgroundScheduler
import sys


def data_engineering(df):
    ##Creating id for each unique player
    df['Player_1']=df['Player_1'].str.lower()
    df['Player_2']=df['Player_2'].str.lower()
    df['Winner']=df['Winner'].str.lower()
    all_players = list(df['Player_1'].unique()) + list(df['Player_2'].unique())


    unique_players = list(set(all_players))
    global player_id_map
    player_id_map = {player: i for i, player in enumerate(unique_players)}


    new_df = df.copy()
    new_df['Player_1_ID'] = new_df['Player_1'].map(player_id_map)
    new_df['Player_2_ID'] = new_df['Player_2'].map(player_id_map)
    new_df['Winner_id'] = new_df['Winner'].map(player_id_map)
    for index, row in new_df.iterrows():
        if row['Winner_id'] == row['Player_1_ID']:
            new_df.at[index, 'Label'] = 1
            new_df.at[index, 'Loser_id'] = row['Player_2_ID']
        if row['Winner_id'] == row['Player_2_ID']:
            new_df.at[index, 'Label'] = 0
            new_df.at[index, 'Loser_id'] = row['Player_1_ID']


    #Creating elo for each player (all of them start with 1500 and the maximum gain from one match is 24
     ##(similiar to chess))
    global elo_players
    elo_players = defaultdict(int)
    global all_elo
    all_elo = defaultdict(lambda: deque())
    df_elo = []
    k = 24
    for w_id, l_id in zip(new_df['Winner_id'], new_df['Loser_id']):

        elo_w = elo_players.get(w_id, 1500)
        elo_l = elo_players.get(l_id, 1500)

        exp_w = 1/(1+10**((elo_l-elo_w)/400))
        exp_l = 1/(1+10**((elo_w-elo_l)/400))

        elo_w += k*(1-exp_w)
        elo_l += k*(0-exp_l)

        df_elo.append(elo_w-elo_l)

    # Update
        elo_players[w_id] = elo_w
        elo_players[l_id] = elo_l

        all_elo[w_id].append(elo_w)
        all_elo[l_id].append(elo_l)
    new_df['Elo_diff'] = df_elo


    ##Creating elo for each player based on their prefmormance on the court (similiar to the previous one)
    global elo_surfaces
    elo_surfaces = defaultdict(lambda: defaultdict(int))
    all_elo_surfaces = defaultdict(lambda: defaultdict(lambda: deque()))
    df_elo = []

    for w_id, l_id, surface in zip(new_df['Winner_id'], new_df['Loser_id'], new_df['Surface']):
        elo_w = elo_surfaces[surface].get(w_id, 1500)
        elo_l = elo_surfaces[surface].get(l_id, 1500)

        exp_w = 1/(1+10**((elo_l-elo_w)/400))
        exp_l = 1/(1+10**((elo_w-elo_l)/400))

        elo_w += k*(1-exp_w)
        elo_l += k*(0-exp_l)
        df_elo.append(elo_w-elo_l)

    # Update
        elo_surfaces[surface][w_id] = elo_w
        elo_surfaces[surface][l_id] = elo_l

        all_elo_surfaces[surface][w_id].append(elo_w)
        all_elo_surfaces[surface][l_id].append(elo_l)

        for s in ["Clay", "Grass", "Hard", "Carpet"]:
            if surface != s:
                all_elo_surfaces[s][w_id].append(elo_surfaces[s].get(w_id, 1500))
                all_elo_surfaces[s][l_id].append(elo_surfaces[s].get(l_id, 1500))

    new_df["ELO_SURFACE_DIFF"] = df_elo
    new_df.drop(['Tournament',"Date",'Series',"Court",'Surface',"Round","Best of",'Player_1','Player_2','Winner','Score','Player_1_ID','Player_2_ID','Winner_id','Loser_id','Pts_1',"Pts_2"],axis=1,inplace=True)
    new_df['Rank_1'] =new_df['Rank_1'].astype(float)
    new_df['Rank_2'] =new_df['Rank_2'].astype(float)
    new_df['Odd_1'] =new_df['Odd_1'].astype(float)
    new_df['Odd_2'] =new_df['Odd_2'].astype(float)



    return new_df


def create_plot():
    nadal = list(all_elo[player_id_map.get('nadal r.')])
    novak = list(all_elo[player_id_map.get('djokovic n.')])
    federer = list(all_elo[player_id_map.get('federer r.')])

    fig = plt.figure(figsize=(8, 4))
    for player in all_elo.keys():
        plt.plot(list(all_elo[player]), marker='.', linewidth=0.1, markersize=1, linestyle='-', color='black')

##מוסיף צבעים לשלושת השחקנים עם האלו הכי גבוה
    plt.plot(nadal, marker='.', linewidth=0.5, markersize=1, linestyle='-', color='blue', label="Rafa Nadal")
    plt.plot(novak, marker='.', linewidth=0.5, markersize=1, linestyle='-', color='red', label="Novak Djokovic")
    plt.plot(federer, marker='.', linewidth=0.5, markersize=1, linestyle='-', color='green', label="Roger Federer")
    plt.title("Elo Ratings Over Time")
    plt.xlabel("Match Number")
    plt.ylabel("Elo Rating")
    plt.legend(loc="lower right")
    plt.draw()
    plt.show()



def lr_spark(df):
    spark = SparkSession \
    .builder \
    .appName("Spark Logistic regression model") \
    .getOrCreate()


    spark_df = spark.createDataFrame(new_df)
    feature_cols = ['Rank_1', 'Rank_2', 'Odd_1', 'Odd_2', 'Elo_diff', 'ELO_SURFACE_DIFF']
    assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
    spark_df = assembler.transform(spark_df)
    spark_train, spark_test = spark_df.randomSplit([0.8, 0.2], seed=42)
    lr = LogisticRegression(featuresCol='features', labelCol='Label')
    lr_model = lr.fit(spark_train)
    predictions = lr_model.transform(spark_test)
    evaluator = MulticlassClassificationEvaluator( labelCol='Label', predictionCol='prediction', metricName='accuracy')
    accuracy = evaluator.evaluate(predictions)
    return lr_model,accuracy



def rf_spark(df):
    spark = SparkSession \
    .builder \
    .appName("Spark Logistic regression model") \
    .getOrCreate()


    spark_df = spark.createDataFrame(new_df)
    feature_cols = ['Rank_1', 'Rank_2', 'Odd_1', 'Odd_2', 'Elo_diff', 'ELO_SURFACE_DIFF']
    assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
    spark_df = assembler.transform(spark_df)
    spark_train, spark_test = spark_df.randomSplit([0.8, 0.2], seed=42)
    rf = RandomForestClassifier(featuresCol='features', labelCol='Label', numTrees=100, seed=42)
    spark_train, spark_test = spark_df.randomSplit([0.8, 0.2], seed=42)
    rf_model = rf.fit(spark_train)
    predictions = rf_model.transform(spark_test)
    evaluator = MulticlassClassificationEvaluator( labelCol='Label', predictionCol='prediction', metricName='accuracy')
    accuracy = evaluator.evaluate(predictions)
    return rf_model,accuracy


def get_an_input_for_prediction(text):
    user_input = input(text)
    return user_input


def use_a_model(model):
    surfaces =["Clay", "Grass", "Hard", "Carpet"]
    surface = None
    Player_1 = None
    while Player_1 not in player_id_map.keys():
        Player_1 = get_an_input_for_prediction("What is the Name of Player 1")
        #lets say we get Roger Federer and we need to make it into federer r.
        Player_1 =f"{Player_1.split()[1].lower()} {Player_1.split()[0][0].lower()}."
        if Player_1 not in player_id_map.keys():
            print("Player not found try again")
    Player_2 = None
    while Player_2 not in player_id_map.keys() or Player_1 == Player_2:
        Player_2 = get_an_input_for_prediction("What is the Name of Player 2")
        #lets say we get Roger Federer and we need to make it into federer r.
        Player_2 =f"{Player_2.split()[1].lower()} {Player_2.split()[0][0].lower()}."

        if Player_2 not in player_id_map.keys():
            print("Player not found try again")
        if Player_2 == Player_1:
            print('Player cannot play himself')
    while surface not in surfaces:
        surface = get_an_input_for_prediction("What is the surface? (Clay,Grass,Hard,Carpet)")
        if surface not in surfaces:
            print('Surface is invalid try again')

    Rank_1 = int(get_an_input_for_prediction("What is player 1 ranking?"))
    Rank_2 = int(get_an_input_for_prediction("What is player 2 ranking"))
    Odd_1 = float(get_an_input_for_prediction("What is player 1 odds?"))
    Odd_2 = float(get_an_input_for_prediction("What is player 2 odds?"))
    Player_1_ID= player_id_map.get(Player_1)
    Player_2_ID= player_id_map.get(Player_2)
    Elo_diff = float(elo_players.get(Player_1_ID)-elo_players.get(Player_2_ID))
    ELO_SURFACE_DIFF=float(elo_surfaces.get(surface).get(Player_1_ID)-elo_surfaces.get(surface).get(Player_2_ID))
    data = [(Rank_1, Rank_2, Odd_1, Odd_2, Elo_diff, ELO_SURFACE_DIFF)]
    columns = ["Rank_1", "Rank_2", "Odd_1", "Odd_2", "Elo_diff", "ELO_SURFACE_DIFF"]
    df = spark.createDataFrame(data, columns)
    assembler = VectorAssembler(inputCols=columns, outputCol="features")
    df_transformed = assembler.transform(df)
    prediction = model.transform(df_transformed)
    if prediction.select('prediction').take(1)[0][0] == 1:
        print(f'Model predict Player 1 to win')
    else:
        print(f'Model predict Player 2 to win')



def data_retrivel():
    consumer = KafkaConsumer(
        'tennis_daily_data',
        bootstrap_servers='localhost:9092',
        group_id=str(uuid.uuid4()),
        auto_offset_reset='earliest',
        enable_auto_commit=True,
        value_deserializer=lambda m: m.decode('utf-8')
    )

# קריאת הנתונים
    data = []
    x=0
    for message in tqdm(consumer):
        if x>1 and data[0]==message.value.split(','):
            break
        x+=1
        row = message.value.split(',')  # פיצול לפי פסיקים
        data.append(row)

    df = pd.DataFrame(data,columns=['Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Player_1', 'Player_2', 'Winner', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2', 'Score']
)
    global new_df
    new_df = data_engineering(df)
    #lr_sklearn,acc_lr_sklearn = lr_sklearn(new_df)
    lr_spark_model,acc_lr_spark = lr_spark(new_df)
   # rf_sklearn,acc_rf_sklearn = random_forest(new_df)
    rf_spark_model,acc_rf_spark = rf_spark(new_df)
    scheduler.add_job(reset_var, 'cron', hour=8, minute=0)
    ui(acc_lr_spark,acc_rf_spark,lr_spark_model,rf_spark_model)
    
def ui(acc_lr_spark,acc_rf_spark,lr_spark_model,rf_spark_model):
    global user_input
    user_input = None
    while True and user_input !=9:
        user_input = input(f'Input 1 To see Elo Graph\n Input 2 To see Models accuracy\n Input 3 To Predict Match on a model\n')
        if int(user_input) == 1:
            create_plot()
        if int(user_input) == 2:
            print(f'The accuracy for Logistic Regression is {acc_lr_spark}\n The accuracy for Random Forest is {acc_rf_spark}')
        if int(user_input) == 3:
            user_input =input(f'Input 1 To use Logistic Regression\n Input 2 To use Random Forest\n' )
            if int(user_input) ==1:
                use_a_model(lr_spark_model)
            if int(user_input) ==2:
                use_a_model(rf_spark_model)
        if int(user_input) == 9:
            return None

def reset_var():
    user_input =9
    scheduler.add_job(data_retrieval)
        

scheduler = BackgroundScheduler()
scheduler.add_job(data_retrivel, 'cron', hour=8, minute=0)
scheduler.start()
data_retrivel()





64781it [00:01, 42041.80it/s]
25/03/25 20:52:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/03/25 20:53:05 WARN TaskSetManager: Stage 0 contains a task of very large size (1063 KiB). The maximum recommended task size is 1000 KiB.
25/03/25 20:53:09 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/03/25 20:53:09 WARN TaskSetManager: Stage 1 contains a task of very large size (1063 KiB). The maximum recommended task size is 1000 KiB.
25/03/25 20:53:10 WARN TaskSetManager: Stage 2 contains a task of very large size (1063 KiB). The maximum recommended task size is 1000 KiB.
25/03/25 20:53:10 WARN TaskSetManager: Stage 3 contains a task of very large size (1063 KiB). The maximum recommended task size is 1000 KiB.
25/03/25 20:53:10 WARN TaskSetManager: Stage 4 contains a task of very large size (1063 KiB). The maximum recommended task size is 1000 KiB.
25/03/25 20:53:11 WARN TaskSetManager: